ASR - Automatic Speech Recognition

Whisper Model is open source model called Whisper, which can be used for multiple regions like translation.

It is build on transformer architecture with encoder and decoder architecture.

In this project, whisper model will be used for voice transcription from Speech to Text since it supports non-english transcription. We will finetune the model with nepali dataset and use it for the Project Jijumuwa.

The whisper-small model has million of parameters.

For this project, the goal is real time speech to text transcibtion which can be done using Faster Whisper.
Another goal to achieve is real time sentiment analysis with conversation.


Finetuning of Whisper Model in nepali language

In [ ]:
pip install evaluate dataset torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.7 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
import evaluate
from transformers import DataCollatorForSeq2Seq

In [ ]:
def prepare_dataset():
    dataset = load_dataset("mozilla-foundation/common_voice_11_0", "ne-NP", trust_remote_code=True)
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_features(batch):
        audio = batch["audio"]
        batch["input_features"] = processor(
            audio["array"],
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features[0]
        batch["labels"] = processor(text=batch["sentence"]).input_ids
        return batch

    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
    dataset = dataset.map(prepare_features, remove_columns=dataset.column_names["train"])
    return dataset, processor

dataset, processor = prepare_dataset()

n_shards.json:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

ne-NP_train_0.tar:   0%|          | 0.00/133k [00:00<?, ?B/s]

ne-NP_dev_0.tar:   0%|          | 0.00/81.9k [00:00<?, ?B/s]

ne-NP_test_0.tar:   0%|          | 0.00/71.7k [00:00<?, ?B/s]

ne-NP_other_0.tar:   0%|          | 0.00/4.80M [00:00<?, ?B/s]

...

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Custom collator for Whisper fine-tuning.

    Steps:
      1. Convert each audio feature array/list into a Torch tensor.
      2. Zero-pad them along the time dimension so they can be stacked.
      3. Use the WhisperProcessor's tokenizer to pad the text labels.
      4. Replace pad_token_id with -100 in labels so they're ignored in the loss.
    """
    processor: Any  # Typically a WhisperProcessor

    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, torch.Tensor]:
        # Separate the audio features (list of arrays) and the labels (list of lists of token IDs).
        input_features = [example["input_features"] for example in features]
        label_ids_list = [example["labels"] for example in features]

        # 1. Convert input_features to torch tensors and measure the max length (time dimension).
        #    Each feature is shape [n_mels, time].
        input_features_tensors = []
        max_time = 0
        for feat in input_features:
            # Convert to float32 torch tensor
            tensor_feat = torch.tensor(feat, dtype=torch.float32)
            input_features_tensors.append(tensor_feat)
            if tensor_feat.shape[-1] > max_time:
                max_time = tensor_feat.shape[-1]

        # 2. Zero-pad these audio tensors along the time dimension to stack them.
        #    We assume the number of mel bins is the same (usually 80 for Whisper).
        batch_size = len(input_features_tensors)
        n_mels = input_features_tensors[0].shape[0]  # e.g. 80
        padded_features = torch.zeros((batch_size, n_mels, max_time), dtype=torch.float32)

        for i, tensor_feat in enumerate(input_features_tensors):
            time_len = tensor_feat.shape[-1]
            padded_features[i, :, :time_len] = tensor_feat

        # 3. Pad labels using the processor's tokenizer to have uniform length across the batch.
        labels = self.processor.tokenizer.pad(
            {"input_ids": label_ids_list},
            padding=True,
            return_tensors="pt",
        ).input_ids

        # 4. Convert tokenizer pad token ID to -100 to ignore label padding in the loss.
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "input_features": padded_features,  # shape [B, n_mels, max_time]
            "labels": labels,                   # shape [B, max_label_len]
        }

In [ ]:
def train_model(dataset, processor):
    """
    Fine-tune the Whisper model with a custom collator handling audio input_features.
    `dataset` is expected to have "train" and "validation" splits, where each example has:
      - "input_features" (mel-spectrogram as an array/list [n_mels, time])
      - "labels" (list of token IDs for the transcript)
    """

    # 1. Load the pre-trained Whisper model
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

    # 2. Define training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir="whisper-nepali",
        per_device_train_batch_size=16,
        gradient_accumulation_steps=2,
        learning_rate=1e-4,
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        fp16=True,
        evaluation_strategy="steps",
        eval_steps=1000,
        save_steps=1000,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
    )

    # 3. Load the WER metric
    wer_metric = evaluate.load("wer")

    # 4. Metrics function
    def compute_metrics(eval_preds):
        """
        eval_preds.predictions are token IDs predicted by the model.
        eval_preds.label_ids are the true token IDs for the labels.
        """
        pred_ids = eval_preds.predictions
        label_ids = eval_preds.label_ids

        # Replace -100 with pad_token_id so we can decode them.
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # Decode predictions and references to strings
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # Calculate WER
        wer_value = wer_metric.compute(predictions=pred_str, references=label_str)
        return {"wer": wer_value}

    # 5. Create the custom data collator
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    # 6. Initialize the Trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        # Do NOT pass `tokenizer` to avoid the FutureWarning. We'll rely on the custom collator.
        tokenizer=None,
    )

    # 7. Train
    trainer.train()

    # 8. Save the model
    trainer.save_model("whisper-nepali-final")

    # Return trainer if you want to evaluate or do something else outside this function
    return trainer


In [ ]:
if __name__ == "__main__":
    # Suppose you have a function that returns a dataset with "train/validation/test" splits
    # Each example has:
    #   example["input_features"] -> mel-spectrogram as a 2D array [n_mels, time]
    #   example["labels"] -> list of token IDs for the transcript
    dataset, processor = prepare_dataset()

    # Fine-tune
    trainer = train_model(dataset, processor)

    # Evaluate on test set if available
    test_results = trainer.evaluate(eval_dataset=dataset["test"])
    print("Test WER:", test_results["eval_wer"])

<ipython-input-33-c1b8a98745dc>:59: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss
